In [72]:
import torch
# x,y 为leaf节点，也就是说，在计算的时候，PyTorch只会保留此节点的梯度值
x = torch.tensor([3.], requires_grad=True)
y = torch.tensor([5.], requires_grad=True)
# a,b均为中间值，在计算梯度时，此部分会被释放掉
a = x + y
b = x * y
c = a * b
# 新建列表，用于存储Hook函数保存的中间梯度值
a_grad = []
def hook_grad(grad):
    a_grad.append(grad)
# register_hook的参数为一个函数
handle = a.register_hook(hook_grad)
c.backward()
# 只有leaf节点才会有梯度值
print('gradient:',x.grad, y.grad, a.grad, b.grad, c.grad)
# Hook函数保留下来的中间节点a的梯度
print('a_grad:', a_grad[0])
# 移除Hook函数
handle.remove()

gradient: tensor([40.]) tensor([24.]) None None None
a_grad: tensor([0.])


/tmp/ipykernel_27122/694724535.py:19: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647378361/work/build/aten/src/ATen/core/TensorBody.h:489.)
  print('gradient:',x.grad, y.grad, a.grad, b.grad, c.grad)


In [73]:
import torch
import torch.nn as nn
# 构建网网络，一个卷积层一个池化层
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 3)
        self.pool1 = nn.MaxPool2d(2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        return x
# 初始化网络
net = Net()
# detach将张量分离
net.conv1.weight[0].detach().fill_(1)
net.conv1.weight[1].detach().fill_(2)
net.conv1.bias.detach().zero_()
# 构建两个列表用于保存信息
fmap_block = []
input_block = []
def forward_hook(module, data_input, data_output):
    fmap_block.append(data_output)
    input_block.append(data_input)
# 注册Hook
net.conv1.register_forward_hook(forward_hook)
# 输入数据
fake_img = torch.ones((1, 1, 4, 4))
output = net(fake_img)
# 观察结果
# 卷积神经网络输出维度和结果
print("output share:{}\noutput value:{}\n".format(output.size(),output))
# 卷积神经网络Hook函数返回的结果
print("feature map share:{}\noutput value:{}\n".format(fmap_block[0].shape,fmap_block[0]))
# 输入的信息
print("input share:{}\ninput value:{}\n".format(input_block[0][0].size(),input_block[0][0]))

output share:torch.Size([1, 2, 1, 1])
output value:tensor([[[[ 9.]],

         [[18.]]]], grad_fn=<MaxPool2DWithIndicesBackward0>)

feature map share:torch.Size([1, 2, 2, 2])
output value:tensor([[[[ 9.,  9.],
          [ 9.,  9.]],

         [[18., 18.],
          [18., 18.]]]], grad_fn=<ConvolutionBackward0>)

input share:torch.Size([1, 1, 4, 4])
input value:tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]])



In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x  
def main():
    img_path = './car.jpg'
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    img = Image.open(img_path)
    img = transform(img)
    img.unsqueeze_(dim=0)
    # 实例化
    net = LeNet()
    PATH = 'cifar_net_10.pth'
    # 将训练好的参数导入
    net.load_state_dict(torch.load(PATH))
    fmap_block = []
    input_block = []
    def forward_hook(module, data_input, data_output):
        fmap_block.append(data_output)
        input_block.append(data_input)
    # 注册Hook
    net.conv1.register_forward_hook(forward_hook)
    net.conv2.register_forward_hook(forward_hook)
    with torch.no_grad():
        outputs = net(img)
        print("conv1 feature map share:{}".format(fmap_block[0].shape))
        print("conv2 feature map share:{}".format(fmap_block[1].shape))
if __name__ == '__main__':
    main()

In [75]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.tensorboard import SummaryWriter

# 定义一个简单的钩子函数，用于可视化中间特征图
def visualize_feature_map(module, input):
    x = input[0]  # 获取输入
    image_grid = torchvision.utils.make_grid(x, normalize=True, scale_each=True)  # 创建图像网格
    writer.add_image("Feature Map", image_grid, global_step=0)  # 添加到 TensorBoard 中

# 创建模型
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.pool(self.conv1(x))
        return x

# 初始化模型和数据
model = MyModel()
writer = SummaryWriter()  # 创建一个 TensorBoard SummaryWriter

# 注册前向传播前的钩子
hook_handle = model.conv1.register_forward_pre_hook(visualize_feature_map)

# 创建一个随机输入
input_data = torch.rand(1, 3, 64, 64)

# 前向传播
output = model(input_data)

# 移除钩子
hook_handle.remove()

# 关闭 SummaryWriter
writer.close()